In [ ]:
!pip install openai datasets

In [ ]:
from datasets import load_dataset

In [ ]:
import openai

openai.api_key = "" # Add your OpenAI API key here!

# MODEL = "gpt-4" # 8K context	$0.03 / 1K tokens	$0.06 / 1K tokens (Expensive!)
MODEL = "gpt-3.5-turbo" # gpt-3.5-turbo	$0.002 / 1K tokens

dataset = load_dataset("banking77")

# Sort the dataset by the length of the customer texts
sorted_data = sorted(dataset['train'], key=lambda x: len(x['text']), reverse=True)

# Extract the longest 5 customer texts
longest_five_texts = [entry["text"] for entry in sorted_data[:5]]

# Print the longest 5 customer texts
for i, text in enumerate(longest_five_texts):
    print(f"Longest Customer Text {i + 1} (Length: {len(text)}): {text}")


def get_completion(prompt_to_gpt, model=MODEL):
    messages = [{"role": "user", "content": prompt_to_gpt}]
    response_from_gpt = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response_from_gpt.choices[0].message["content"]

for i in range(len(longest_five_texts)):
    prompt_1 = f"""
  What is the sentiment of the following comments, 
  which is delimited with triple backticks?

  Give your answer as a single word, either "positive" \
  or "negative".

  Review text: '''{longest_five_texts[i]}'''
  """
    sentiment = get_completion(prompt_1)
    print(i, sentiment, "\n")

    prompt_2 = f"""
  You are a customer service AI assistant.
  Your task is to send an email reply to a valued customer.
  Given the customer email delimited by ```, \
  Generate a reply to thank the customer for their review.
  If the sentiment is positive or neutral, thank them for \
  their review.
  If the sentiment is negative, apologize and suggest that \
  they can reach out to customer service. 
  Make sure to use specific details from the review.
  Write in a concise and professional tone.
  Sign the email as `AI customer agent`.
  Customer review: ```{longest_five_texts[i]}```
  Review sentiment: {sentiment}
  """
    response = get_completion(prompt_2)
    print(i, response, "\n")
